# Graphical Lasso Notebook

In [14]:
import sys
import csv
import pandas as pd
import numpy as np
import scipy as sp
import igraph as ig
import matplotlib.pyplot as plt
from sklearn.covariance import GraphicalLassoCV

In [16]:
def main(filename):
	#データを読み込む
	fp = open(filename)
	dat1 = list(csv.reader(fp))

	#最初の行をとる
	feature_names = dat1[0]
	dat1 = dat1[1:]
	dat2 = [list(map(int, row)) for row in dat1]
	X = ltonpa(dat2)
    
	#正規化
	X = sp.stats.zscore(X, axis=0)
    
	model = GraphicalLassoCV(alphas=4, cv=2, verbose=True)
	#model = GraphicalLassoCV(alphas=, verbose=True)
	model.fit(X)
	cov_ = model.covariance_
	prec_ = model.precision_

	##グラフデータ生成と表示
	graph_data = glasso_graph_make(model, feature_names, threshold=0.6)
	#graph_data
	plt.show()

In [17]:
#グラフ生成関数
def glasso_graph_make(model, feature_names, threshold):
	#分散共分散行列を取得 (Get varience-covarience matrix)
	covariance_matrix = model.covariance_

	#分散共分散行列を相関行列に変換
	diagonal = np.sqrt(covariance_matrix.diagonal())
	correlation_matrix = ((covariance_matrix.T / diagonal).T) / diagonal

	#グラフ表示のために体格成分が0の行列を生成する。
	correlation_matrix_diag_zero = correlation_matrix - np.diag(np.diag(correlation_matrix))
	df_graph_data = pd.DataFrame(index=feature_names, columns=feature_names, data=correlation_matrix_diag_zero.tolist())

	#グラフ生成準備
	graph_data = ig.Graph()
	graph_data.add_vertices(len(feature_names))
	graph_data.vs["feature_names"] = feature_names
	graph_data.vs["label"] = graph_data.vs["feature_names"]
	visual_style = {}
	edge_width_list = []
	edge_color_list = []

	#グラフ生成
	for target_index in range(len(df_graph_data.index)):
		for target_column in range(len(df_graph_data.columns)):
			if target_column >= target_index:
				graph_data_abs_element = df_graph_data.iloc[target_index, target_column]
				if abs(graph_data_abs_element) >= threshold:
					edge = [(target_index, target_column)]
					graph_data.add_edges(edge)
					edge_width_list.append(abs(graph_data_abs_element)*10)
					if graph_data_abs_element > 0:
						edge_color_list.append("red")
					else:
						edge_color_list.append("blue")
	
	visual_style["edge_width"] = edge_width_list
	visual_style["edge_color"] = edge_color_list
	draw_graph = ig.plot(graph_data, **visual_style, vertex_size=50, bbox=(500,500), vertex_color="skyblue", layout="circle", margin=50)
	draw_graph
	return draw_graph

In [18]:
#List to "numpy.array"
def ltonpa(dat1):
	npa = np.array(dat1)
	return npa

In [19]:
main("group9.csv")

[GraphicalLassoCV] Done refinement  1 out of 4:   0s
[GraphicalLassoCV] Done refinement  2 out of 4:   0s
[GraphicalLassoCV] Done refinement  3 out of 4:   0s
[GraphicalLassoCV] Done refinement  4 out of 4:   0s


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
